In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

from torch.autograd import Variable
from torch import optim
from PIL import Image

In [2]:
""" Load the train dataset """
# 数据预处理，转化为 Tensor
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='', train=True, download=False, transform=transform)
trainloader=torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

In [3]:
""" Load the test dataset """
# 数据预处理，转化为 Tensor
transform = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.CIFAR10(root='', train=False, download=False, transform=transform)
testloader=torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True, num_workers=4)

In [4]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


In [5]:
# alex = models.alexnet(pretrained=True)
# alex.features = alex.features[:-1]

In [6]:
''' define a network structure '''
cfg = [64, 'M', 128, 'M', 256, 256]    # 自己定义一个网络
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.features = self._make_layers(cfg)
        self.linear = torch.nn.Sequential(
            nn.Linear(256, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )
        
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=8, stride=1)]
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256)
#         print(x.shape)

        x = self.linear(x)
        return x
net = mynet()

# if I have a GPU, use it.
net.to(device)

mynet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): AvgPool2d(kernel_size=8, stride=1, padding=0)
  )
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [7]:
''' train this model '''
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.005,momentum=0.9)
epoch_num = 20

for epoch in range(epoch_num):
    print('-------- epoch {} --------'.format(epoch))
    running_loss = 0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data
        inputs,labels = Variable(inputs).to(device), Variable(labels).to(device)
        
        optimizer.zero_grad()    # 梯度清零
        outputs = net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i%200 == 0:
            print(loss)

-------- epoch 0 --------
tensor(2.3191)


/Users/coder_cxk/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(2.2874)
tensor(2.2833)
tensor(1.9107)
tensor(2.2950)
tensor(1.8433)
tensor(1.7897)
tensor(1.9516)
tensor(1.5367)
tensor(1.7986)
tensor(1.9460)
tensor(2.5040)
tensor(2.1792)
tensor(1.8521)
tensor(1.8066)
tensor(1.6193)
-------- epoch 1 --------
tensor(1.7332)
tensor(1.6383)
tensor(1.3384)
tensor(1.4310)
tensor(1.3753)
tensor(1.6547)
tensor(1.5276)
tensor(1.2767)
tensor(1.4108)
tensor(1.5671)
tensor(1.8481)
tensor(1.2405)
tensor(1.1600)
tensor(1.5078)
tensor(1.7934)
tensor(1.9120)
-------- epoch 2 --------
tensor(1.1727)
tensor(1.4695)
tensor(1.3331)
tensor(1.1416)
tensor(1.1983)
tensor(1.3475)
tensor(1.1891)
tensor(1.0962)
tensor(1.3839)
tensor(1.4674)
tensor(1.1393)
tensor(0.8101)
tensor(1.5334)
tensor(0.9010)
tensor(1.6604)
tensor(1.4308)
-------- epoch 3 --------
tensor(1.1357)
tensor(1.0656)
tensor(0.7618)
tensor(0.7622)
tensor(1.1016)
tensor(0.8567)
tensor(1.1560)
tensor(1.0401)
tensor(1.0346)
tensor(0.9201)
tensor(0.9067)
tensor(0.9063)
tensor(0.8581)
tensor(0.9504)
tensor(

In [8]:
''' test the dataset '''
tot_case, correct_case = 0, 0
for i, data in enumerate(testloader,0):
    inputs, labels = data
    inputs,labels = Variable(inputs).to(device), Variable(labels).to(device)
    outputs = net(inputs)
    for i in range(len(outputs)):
        out, label = outputs[i], labels[i].long()
        out_max = torch.max(out, 0)[1].long()
        tot_case += 1
        if torch.equal(out_max, label) == True:
            correct_case += 1
print('accuracy is {}%'.format(correct_case/tot_case*100))

accuracy is 75.6%


In [ ]:
''' save the model '''
torch.save(net.state_dict(), 'params.pkl')